In [1]:
# Check if GPU is available
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-04-30 17:07:25.226582: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 17:07:25.940543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1


2024-04-30 17:07:26.516801: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 17:07:26.555071: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 17:07:26.555294: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
%load_ext tensorboard

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, RandomContrast, RandomFlip, RandomRotation, RandomBrightness, RandomTranslation
from pathlib import Path
import matplotlib.pyplot as plt
import pydot
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras
import keras_tuner as kt

In [22]:
# Main data dir
train_data_dir = Path('train/').with_suffix('')
test_data_dir = Path('test/').with_suffix('')

# Parameters
batch_size = 32
image_size = (150, 150)
num_classes = 15
epochs = 20

In [5]:
plant_labels = [
    "Ulmus carpinifolia",
    "Acer",
    "Salix aurita",
    "Quercus",
    "Alnus incana",
    "Betula pubescens",
    "Salix alba 'Sericea'",
    "Populus tremula",
    "Ulmus glabra",
    "Sorbus aucuparia",
    "Salix sinerea",
    "Populus",
    "Tilia",
    "Sorbus intermedia",
    "Fagus silvatica"
]
plant_labels = sorted(plant_labels)
print(plant_labels)

In [23]:
# Data loading and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255, # rescale data
    validation_split=0.2
)  # 20% validation split

# Load and preprocess data
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=42
    )

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    seed=42
    )

test_generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    seed=42
)

Found 720 images belonging to 15 classes.
Found 180 images belonging to 15 classes.
Found 225 images belonging to 15 classes.


In [7]:
inv_map_train = {v: k for k, v in train_generator.class_indices.items()}
inv_map_test = {v: k for k, v in test_generator.class_indices.items()}

In [10]:
# Augment Data

data_augmentation = tf.keras.Sequential([
  RandomRotation(0.3),
  RandomContrast(0.3),
  RandomBrightness(0.3),
  RandomTranslation(0.3,0.3)
])

2024-04-30 17:07:27.212298: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 17:07:27.212532: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 17:07:27.212682: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [11]:

# Model creation
def create_model(hp):
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model = Sequential()
    # model.add(data_augmentation)
    # model = Sequential([
    #     data_augmentation,
    #     Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     Conv2D(64, (3, 3), activation='relu'),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     # Conv2D(128, (3, 3), activation='relu'),
    #     # MaxPooling2D(pool_size=(2, 2)),
    #     Flatten(),
    #     # Dense(256, activation='relu'),
    #     Dense(units=hp_units, activation='relu'),
    #     Dense(num_classes, activation='softmax')
    # ])
    # layers = hp.Int("num_layers", 2, 6)
    # model.add(RandomRotation(0.3))
    # model.add(RandomContrast(0.3))
    for i in range(1, hp.Int("num_layers", 2, 6)):
        model.add(Conv2D(hp.Int(f"filters_{i}", 32, 128, step=32), (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(units=hp_units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [12]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [13]:
tuner = kt.Hyperband(create_model,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='runs',
                     project_name='leaf_class')


Reloading Tuner from runs/leaf_class/tuner0.json


In [15]:
tuner.search(train_generator, epochs=20, validation_data=test_generator, callbacks=[stop_early])

Trial 26 Complete [00h 01m 55s]
val_accuracy: 0.8222222328186035

Best val_accuracy So Far: 0.8222222328186035
Total elapsed time: 00h 21m 01s


In [16]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of layers is {best_hps.get('num_layers')}. The optimal number of units in the densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



The hyperparameter search is complete. The optimal number of layers is 6. The optimal number of units in the densely-connected
layer is 480 and the optimal learning rate for the optimizer
is 0.001.



In [19]:
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(train_generator, validation_data=test_generator)

29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 227ms/step - accuracy: 0.0728 - loss: 2.6728 - val_accuracy: 0.2711 - val_loss: 2.1698


In [ ]:
# Model training
# history = model.fit(train_generator, epochs=epochs, validation_split=0.2)

In [20]:
hypermodel.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_rotation_2               │ (None, 150, 150, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast_2               │ (None, 150, 150, 3)    │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 148, 148, 96)   │         2,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 74, 74, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 72, 72, 32)     │        27,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 36, 36, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 34, 34, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 17, 17, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 15, 15, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 5, 5, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 480)            │       246,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │         7,215 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,571,279 (5.99 MB)

 Trainable params: 523,759 (2.00 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,047,520 (4.00 MB)

In [ ]:
hypermodel.save('hypermodel_10.keras')

In [ ]:
dot_img_file = 'model_10.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
%matplotlib inline

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

In [ ]:
def preprocess_image(img_path, target_size=(150,150)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

In [ ]:
test_image = preprocess_image('hahaha.jpg')
model.evaluate(test_generator)


In [ ]:
idx = np.argmax(model.predict(test_image))
inv_map[idx]